In [ ]:
-- Create the database
CREATE DATABASE fraud_detection_db;

-- Connect to the database
\c fraud_detection_db;

-- Create the transactions table
CREATE TABLE transactions (
    time DOUBLE PRECISION,
    v1 DOUBLE PRECISION,
    v2 DOUBLE PRECISION,
    v3 DOUBLE PRECISION,
    v4 DOUBLE PRECISION,
    v5 DOUBLE PRECISION,
    v6 DOUBLE PRECISION,
    v7 DOUBLE PRECISION,
    v8 DOUBLE PRECISION,
    v9 DOUBLE PRECISION,
    v10 DOUBLE PRECISION,
    v11 DOUBLE PRECISION,
    v12 DOUBLE PRECISION,
    v13 DOUBLE PRECISION,
    v14 DOUBLE PRECISION,
    v15 DOUBLE PRECISION,
    v16 DOUBLE PRECISION,
    v17 DOUBLE PRECISION,
    v18 DOUBLE PRECISION,
    v19 DOUBLE PRECISION,
    v20 DOUBLE PRECISION,
    v21 DOUBLE PRECISION,
    v22 DOUBLE PRECISION,
    v23 DOUBLE PRECISION,
    v24 DOUBLE PRECISION,
    v25 DOUBLE PRECISION,
    v26 DOUBLE PRECISION,
    v27 DOUBLE PRECISION,
    v28 DOUBLE PRECISION,
    amount DOUBLE PRECISION,
    class INTEGER
);

-- Import data into the transactions table (adjust file path accordingly)
-- \\COPY transactions FROM 'path/to/creditcard.csv' DELIMITER ',' CSV HEADER;

-- Query: Total transactions and fraud distribution
SELECT 
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN class = 1 THEN 1 ELSE 0 END) AS fraudulent_transactions,
    SUM(CASE WHEN class = 0 THEN 1 ELSE 0 END) AS non_fraudulent_transactions
FROM transactions;

-- Query: Fraudulent transactions by hour of day
SELECT
    (FLOOR(time / 3600)::INT % 24) AS hour_of_day,
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN class = 1 THEN 1 ELSE 0 END) AS fraudulent_transactions
FROM transactions
GROUP BY hour_of_day
ORDER BY fraudulent_transactions DESC;

-- Query: Categorize transactions based on amount
SELECT 
    CASE 
        WHEN amount <= 50 THEN 'Small'
        WHEN amount > 50 AND amount <= 200 THEN 'Medium'
        WHEN amount > 200 AND amount <= 1000 THEN 'Large'
        ELSE 'Very Large'
    END AS amount_category,
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN class = 1 THEN 1 ELSE 0 END) AS fraudulent_transactions
FROM transactions
GROUP BY amount_category
ORDER BY fraudulent_transactions DESC;

-- Query: High-risk hours with most fraud
SELECT 
    (FLOOR(time / 3600)::INT % 24) AS hour_of_day,
    COUNT(*) AS total_transactions,
    SUM(CASE WHEN class = 1 THEN 1 ELSE 0 END) AS fraudulent_transactions,
    ROUND((SUM(CASE WHEN class = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 2) AS fraud_percentage
FROM transactions
GROUP BY hour_of_day
HAVING SUM(CASE WHEN class = 1 THEN 1 ELSE 0 END) > 0
ORDER BY fraud_percentage DESC;

-- Query: Average transaction amount for fraudulent vs non-fraudulent
SELECT 
    class,
    ROUND(AVG(amount), 2) AS avg_transaction_amount,
    COUNT(*) AS total_transactions
FROM transactions
GROUP BY class;

-- Query: Detect rapid transactions (possible fraud)
SELECT 
    time,
    amount,
    class,
    LEAD(time) OVER (ORDER BY time) - time AS time_diff
FROM transactions
WHERE class = 1
ORDER BY time_diff ASC
LIMIT 10;
